In [76]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.utils import class_weight

In [77]:
DATA_DIR = r'C:\Users\melis\SpiralHandPD_model\Spiral_HandPD'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [78]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [79]:
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [80]:
train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 295 images belonging to 2 classes.


In [81]:
val_gen = val_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 73 images belonging to 2 classes.


In [82]:
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

Class weights: {0: np.float64(2.543103448275862), 1: np.float64(0.6223628691983122)}


In [83]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

In [84]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [85]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    class_weight=class_weights
)

model.save("DenseNet121.h5")

c:\Users\melis\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4950 - loss: 1.2850 - val_accuracy: 0.6575 - val_loss: 0.5345
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 737ms/step - accuracy: 0.7178 - loss: 0.6383 - val_accuracy: 0.8904 - val_loss: 0.3492
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 682ms/step - accuracy: 0.7658 - loss: 0.5337 - val_accuracy: 0.8767 - val_loss: 0.3533
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 695ms/step - accuracy: 0.7388 - loss: 0.4831 - val_accuracy: 0.9178 - val_loss: 0.3037
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 784ms/step - accuracy: 0.7095 - loss: 0.5665 - val_accuracy: 0.8767 - val_loss: 0.2434
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 711ms/step - accuracy: 0.8117 - loss: 0.4186 - val_accuracy: 0.9726 - val_loss: 0.2450
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 701ms/step - accuracy: 0.8126 - loss: 0.4518 - val_accuracy: 0.8630 - val_loss: 0.2355
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 731ms/step - accuracy: 0.7744 - loss: 0.4260 - val_accuracy: 0.93

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(val_gen)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true = val_gen.classes

print(classification_report(y_true, y_pred_labels, target_names=val_gen.class_indices.keys()))

3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step  
[[12  2]
 [ 0 59]]
                precision    recall  f1-score   support

 SpiralControl       1.00      0.86      0.92        14
SpiralPatients       0.97      1.00      0.98        59

      accuracy                           0.97        73
     macro avg       0.98      0.93      0.95        73
  weighted avg       0.97      0.97      0.97        73



In [87]:
from tensorflow.keras.preprocessing import image

In [88]:
img_pathControl = r'C:\Users\melis\SpiralHandPD_model\exampleControl.png'
img_pathPatient = r'C:\Users\melis\SpiralHandPD_model\examplePatient.png'
imgControl = image.load_img(img_pathControl, target_size=(224, 224))
imgPatient = image.load_img(img_pathPatient, target_size=(224, 224))

In [89]:
imgC_array = image.img_to_array(imgControl)
imgC_array = np.expand_dims(imgC_array, axis=0)
imgC_array = imgC_array / 255.0

In [90]:
imgP_array = image.img_to_array(imgPatient)
imgP_array = np.expand_dims(imgP_array, axis=0)
imgP_array = imgP_array / 255.0

In [91]:
from tensorflow.keras.models import load_model

best_model = load_model(r'C:\Users\melis\SpiralHandPD_model\DenseNet121.h5')
pred1 = best_model.predict(imgC_array)
pred2 = best_model.predict(imgP_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


In [92]:
predicted_class = np.argmax(pred1)
class_labels = list(train_gen.class_indices.keys())
print(f"1. için tahmin edilen sınıf: {class_labels[predicted_class]}")

1. için tahmin edilen sınıf: SpiralControl


In [93]:
predicted_class = np.argmax(pred2)
class_labels = list(train_gen.class_indices.keys())
print(f"2. için tahmin edilen sınıf: {class_labels[predicted_class]}")

2. için tahmin edilen sınıf: SpiralPatients
